trying to use the example from 
[http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/)

**biome dataset**:  
denote number of data  
mesa size:  10015  
forest size:  10047  
desert size:  10106  
jungle size:  10238  
eh size:  7364  
training size: 42993  
test size: 4777  
total size:  47770  
Batch size: 5

**animal dataset:**  
denote number of data  
None    label:  0  size:  577  
Pig     label:  1  size:  1644  
Chicken label:  2  size:  597  
Cow     label:  3  size:  456  
training size: 2946   
test size: 328   
total size: 3274   

**weather dataset:**  
label:  0  size:  1196  
label:  0  size:  1309  
label:  1  size:  684  
label:  2  size:  182  
training size: 3033   
test size: 338   
total size: 3371   

In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib as plt
import glob
import numpy as np
from PIL import Image
import mltools as ml

In [2]:
# labels ={"mesa":0, "forest":1, "desert":2,"jungle":3,"eh":4}
labels = {"none":0, "pig":1, 'chicken':2, 'cow':3}
# labels = {"clear_or_normal":0, "rain":1, "thunder":2}
tfrecords_train_filename = '/notebooks/Minecraft-AI/mc-data/pig_reg/loaded_data/none_pig_chicken_cow_train.tfrecords'
tfrecords_test_filename='/notebooks/Minecraft-AI/mc-data/pig_reg/loaded_data/none_pig_chicken_cow_test.tfrecords'

In [3]:
files_paths = [
    ('/notebooks/Minecraft-AI/mc-data/pig_reg/no_pig/**/*.jpg', "0"),
    ('/notebooks/Minecraft-AI/mc-data/pig_reg/pig/**/*.jpg', "1"),
    ('/notebooks/Minecraft-AI/mc-data/pig_reg/chicken_rgb/*.jpg', "2"),
     ('/notebooks/Minecraft-AI/mc-data/pig_reg/cow_rgb/*.jpg', "3")
]

In [4]:
filename_pairs=[]
for file_name, label in files_paths:
    files = glob.glob(file_name)
    file_pairs = zip(files, label*len(files))
    print "label: ", label, " size: ", len(files)
    filename_pairs += file_pairs

label:  0  size:  577
label:  1  size:  1644
label:  2  size:  597
label:  3  size:  456


In [5]:
np.random.shuffle(filename_pairs)
portion=int(0.9*len(filename_pairs))
train_pairs = filename_pairs[:portion]
test_pairs = filename_pairs[portion:]

In [6]:
print train_pairs[:10]

[('/notebooks/Minecraft-AI/mc-data/pig_reg/pig/pig_rgb/pig_1175.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/pig/pig_forest_eh/pig_709.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/pig/pig_rgb/pig_459.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/pig/pig_rgb_2/pig_324.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/pig/pig_rgb/pig_804.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/chicken_rgb/chicken_677.jpg', '2'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/cow_rgb/cow_308.jpg', '3'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/no_pig/fence_rgb_2/fence_337.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/pig/pig_forest_eh/pig_212.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/pig_reg/pig/pig_forest_eh/pig_714.jpg', '1')]


In [7]:
print "training size:", len(train_pairs),"  "
print "test size:", len(test_pairs),"  "
print "total size:", len(filename_pairs),"  "

training size: 2946   
test size: 328   
total size: 3274   


In [8]:

train_writer = tf.python_io.TFRecordWriter(tfrecords_train_filename)
test_writer = tf.python_io.TFRecordWriter(tfrecords_test_filename)

In [9]:
# helper functions
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [10]:
def write_in_tfrecord(filename_pairs, writer):
    for img_path, label in filename_pairs:

        img = np.array(Image.open(img_path))
        
        # The reason to store image sizes was demonstrated
        # in the previous example -- we have to know sizes
        # of images to later read raw serialized string,
        # convert to 1d array and convert to respective
        # shape that image used to have.
        height = img.shape[0]
        width = img.shape[1]

        # Put in the original images into array
        # Just for future check for correctness
#         original_images.append((img,label))

        img_raw = img.tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'image_raw': _bytes_feature(img_raw),
            'label': _int64_feature(int(label))}))

        writer.write(example.SerializeToString())

In [11]:
write_in_tfrecord(train_pairs,train_writer)
train_writer.close()

In [12]:
write_in_tfrecord(test_pairs,test_writer)
test_writer.close()